In [35]:
# 筛选10条json作为实验pipeline测试
path = '../outputs/regenerated_sentences.json'
import json
import random
import os
import shutil
import re

with open(path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# 随机选择10条
random.seed(42)
random.shuffle(data)
data = data[:10]
# 生成json文件

output_path = '../outputs/10_regenerated_sentences.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [1]:
import json
import torch
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration
from qwen_vl_utils import process_vision_info
import gc

image_path = "../images/train2017/000000407466.jpg"

local_model_path = "../models/Qwen2.5-VL-3B/snapshots/1b989f2c63999d7344135894d3cfa8f494116743"
processor = AutoProcessor.from_pretrained(local_model_path)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    local_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


prompt = f"Describe the image in 100 words, including objects, attributes, and positional relationships."
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": prompt},
        ],
    }
]
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(text=[text], images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt").to("cuda")

# generated_ids = model.generate(**inputs, max_new_tokens=100,temperature=0.9)
# output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0].strip()
# print(output_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
print(model)

Qwen2_5_VLForConditionalGeneration(
  (visual): Qwen2_5_VisionTransformerPretrainedModel(
    (patch_embed): Qwen2_5_VisionPatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2_5_VLVisionBlock(
        (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
        (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
        (attn): Qwen2_5_VLVisionSdpaAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): Qwen2_5_VLMLP(
          (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
          (up_proj): Linear(in_features=1280, out_features=3420, bias=True)
          (down_proj): Linear(in_features=3420, out_features=1280, bias=True)
          (act_fn): SiLU()
        )
      )
    )
    (merger): Qwen2_5_VLPatchMerger

In [29]:
prompt_length = inputs["input_ids"].shape[1]
print(prompt_length)
generated_ids = model.generate(**inputs, max_new_tokens=50,temperature=1.0, do_sample=True,top_k=20,top_p=0.8)
output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0].strip()
print(len(output_text))
output_text = output_text[prompt_length:]
print(output_text)

432
221



In [34]:
# 获取 prompt 的 token 数量
prompt_length = inputs["input_ids"].shape[1]

# 生成新内容
generated_ids = model.generate(
    **inputs,
    max_new_tokens=50,
    temperature=1.0,
    do_sample=True,
    top_k=20,
    top_p=0.8
)

# 去掉 prompt 部分，只保留新生成的 token
new_tokens = generated_ids[0][prompt_length:]

# 解码生成部分的 token
output_text = processor.decode(new_tokens, skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()

print(output_text)


There is a rich variety of lunch in the container.


In [1]:
import openai
import base64
import logging
import requests
import re

def validate_sentence_gpt4o(image_path, sentence, api_key):
    openai.api_key = api_key

    # 编写 prompt
    prompt = f"""Determine if the following description is accurate for the image: '{sentence}'.
Return only a single integer (0-100) representing the confidence score, with no other text.
Examples:
- Description: 'A dog runs in a park.' Image shows a dog running. Output: 95
- Description: 'The sky is red.' Image shows a blue sky. Output: 10
- Description: 'A cat sleeps on a couch.' Image shows a cat sleeping. Output: 90
Your output:"""

    # 读取图片为 base64 格式
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode("utf-8")

    # 调用 OpenAI GPT-4o
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                    {"type": "text", "text": prompt}
                ]
            }
        ],
        max_tokens=5,
        temperature=0.7
    )

    # 解析输出
    output_text = response["choices"][0]["message"]["content"].strip()
    print(f"GPT-4o 输出文本: {output_text}")

    # 提取评分数字
    match = re.search(r'\d+', output_text)
    if match:
        score = float(match.group())
        score = max(0, min(100, score))
    else:
        score = 0
        logging.warning(f"无效输出: '{output_text}' for sentence: '{sentence}'")

    return score


score = validate_sentence_gpt4o("your_image.jpg", "A cat sleeps on a couch.", api_key="sk-xxx")
print("GPT-4o 给出的评分：", score)


ModuleNotFoundError: No module named 'openai'